## Import Dependencies

In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import LLMChain
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

#Playwright
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.
)

#Python Tools
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

#Files ToolKit
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

## Load LLM

In [2]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
model_path = "../../../../../Models/llama-2-7b-chat.ggmlv3.q8_0.bin"
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
    callback_manager=callback_manager,
    verbose=True,
)

llama.cpp: loading model from ../../../../../Models/llama-2-7b-chat.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 7130.73 MB (+  256.00 MB per state)
llama_new_context_with_model: kv self size  =  25

## Load Tools

In [3]:
async_browser = create_async_playwright_browser()
scrape_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
scraping_tools = scrape_toolkit.get_tools()

In [4]:
tools_by_name = {tool.name: tool for tool in scraping_tools}
print(tools_by_name.keys())
navigate_tool = tools_by_name["navigate_browser"]
extract_text = tools_by_name["extract_text"]

dict_keys(['click_element', 'navigate_browser', 'previous_webpage', 'extract_text', 'extract_hyperlinks', 'get_elements', 'current_webpage'])


In [5]:
files_tools = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()
read_tool, write_tool, list_tool = files_tools

In [6]:
DuckDuckGoSearch = DuckDuckGoSearchRun()
PythonTool =PythonREPLTool()

tools = [
    Tool(
        name="DuckDuckGo",
        func=DuckDuckGoSearch.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Read directory",
        func=read_tool.run,
        description="Read files within a directory",
    ),
    Tool(
        name="Write directory",
        func=write_tool.run,
        description="Write files to a directory",
    ),
    Tool(
        name="Extract text from webpage",
        func=extract_text.run,
        description="Extracts text from a webpage",
    ),
    extract_text,
    navigate_tool, PythonTool, 
]

In [7]:
prefix = """Answer the following questions as best you You have access to the following tools:"""
suffix = """Begin! Remember to be as creative as you can"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [8]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [10]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [11]:
agent_executor.run("""write a bio for a person in the style of Bruce Wyane, the world's greatest detective. The bio has to aligned to the person/entity in question. Use the example bio as a reference

Use his linkedin profile: https://www.linkedin.com/in/hlumisa-mazomba-4068b23a/""")



> Entering new AgentExecutor chain...

Thought: this should be fun
Action: Read directory to find out more about the person

KeyboardInterrupt: 